## 9.

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784", version=1)
X = mnist['data']
y = mnist['target']
X_train = X[:50000]
y_train = y[:50000]
X_test = X[60000:]
y_test = y[60000:]
X_valid = X[50000:60000]
y_valid = y[50000:60000]

In [2]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

In [3]:
import joblib
rnd_forest = joblib.load("rnd_forest_best.xz")
extra_tree = joblib.load("extra_tree_best.xz")
svc = joblib.load("svc_search_best.xz")

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

random forest and extra tree models loaded above are already trained with standard training set. But svc is not trained with the whole dataset and must be re-trained.

In [6]:
svc.fit(X_train_std, y_train)

[LibSVM]

SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=42, shrinking=True, tol=0.001,
    verbose=True)

In [7]:
X_valid_std = scaler.transform(X_valid)
valid_preds_svc = svc.predict(X_valid_std)
valid_preds_forest = rnd_forest.predict(X_valid_std)
valid_preds_extra_tree = extra_tree.predict(X_valid_std)

In [33]:
import numpy as np
validation_preds = np.c_[valid_preds_svc, valid_preds_forest, valid_preds_extra_tree].astype(np.float32)

In [34]:
from sklearn.linear_model import LogisticRegression
blender_logistic = LogisticRegression(multi_class="multinomial", max_iter=2000, C=1e+3, solver="lbfgs")
blender_logistic.fit(validation_preds, y_valid)

LogisticRegression(C=1000.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_valid, blender_logistic.predict(validation_preds))

0.9616

In [36]:
scaler_blender = StandardScaler()
validation_preds_std = scaler_blender.fit_transform(validation_preds)

In [65]:
from sklearn.svm import SVC
blender_svc_rbf = SVC(kernel="rbf", gamma="scale", random_state=42, C=15)
blender_svc_rbf.fit(validation_preds_std, y_valid)

SVC(C=15, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [66]:
accuracy_score(y_valid, blender_svc_rbf.predict(validation_preds_std))

0.9749

Voting classifer performs better!s
